## Data Warehouse

__Some Questions for the datawarehouse to answer:__

1. Which program has the most professors? Most classes offered?
2. Which course has the most classes offered?
3. What term are the most classes offered?
4. Which classes leave the most seats open?

__Import Packages__

In [1]:
%load_ext sql
import pandas as pd
import sqlite3

In [2]:
%sql sqlite:///CourseDataWarehouse.db

'Connected: @CourseDataWarehouse.db'

In [3]:
%%sql
ATTACH DATABASE 'CourseData.db' as CourseDB;

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [4]:
%%sql
SELECT * FROM Program
LIMIT 5;

 * sqlite:///CourseDataWarehouse.db
Done.


ProgramID,ProgramName,ProgramCode
1,Asian Studies,AN
2,Business,BU
3,Black Studies,BL
4,Bioengineering,BEN
5,Arabic,AR


__Create Tables__

In [5]:
%%sql

DROP TABLE IF EXISTS ProfessorDimension;
CREATE TABLE ProfessorDimension (
    ProfessorID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL
);

DROP TABLE IF EXISTS LocationDimension;
CREATE TABLE LocationDimension (
    LocationID INTEGER PRIMARY KEY,
    Location TEXT NOT NULL
);

DROP TABLE IF EXISTS ProgramDimension;
CREATE TABLE ProgramDimension (
    ProgramID INTEGER PRIMARY KEY,
    ProgramName TEXT NOT NULL,
    ProgramCode TEXT NOT NULL
);

DROP TABLE IF EXISTS TimecodeDimension;
CREATE TABLE TimecodeDimension (
    TimecodeDimensionID INTEGER PRIMARY KEY,
    Day TEXT, 
    StartTime TEXT NOT NULL,
    EndTime TEXT NOT NULL
);

DROP TABLE IF EXISTS CatalogCourseDimension;
CREATE TABLE CatalogCourseDimension (
    CatalogCourseID INTEGER PRIMARY KEY,
    CatalogYear TEXT NOT NULL,
    CatalogID TEXT NOT NULL,
    ProgramID TEXT,
    Credits REAL,
    Prereqs TEXT,
    Coreqs TEXT,
    Fees TEXT,
    Attributes TEXT,
    Description TEXT
);

DROP TABLE IF EXISTS ClassFactTable;
CREATE TABLE ClassFactTable (
    ClassID INTEGER,
    ProfessorID INTEGER,
    ProgramID INTEGER,
    LocationID INTEGER,
    TimecodeDimensionID INTEGER,
    CatalogCourseID INTEGER,
    Term TEXT NOT NULL,
    CRN INTEGER NOT NULL,
    Section TEXT NOT NULL,
    Title TEXT,
    Credits INTEGER,
    Capacity INTEGER NOT NULL,
    Actual INTEGER NOT NULL,
    Remaining INTEGER NOT NULL,
    FOREIGN KEY (ProfessorID) REFERENCES ProfessorDimension (ProfessorID),
    FOREIGN KEY (ProgramID) REFERENCES ProgramDimension (ProgramID),
    FOREIGN KEY (LocationID) REFERENCES LocationDimension (LocationID),
    FOREIGN KEY (TimecodeDimensionID) REFERENCES MeetingDimension (TimecodeDimensionID),
    FOREIGN KEY (CatalogCourseID) REFERENCES CatalogCourseDimension (CatalogCourseID)
);

 * sqlite:///CourseDataWarehouse.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

### __Populate Tables:__

In [6]:
%%sql
DELETE FROM ProfessorDimension;
INSERT INTO ProfessorDimension(Name)
    SELECT DISTINCT Name
    FROM CourseDB.Professor;

 * sqlite:///CourseDataWarehouse.db
0 rows affected.
1095 rows affected.


[]

In [7]:
%%sql
DELETE FROM LocationDimension;
INSERT INTO LocationDimension(Location)
    SELECT DISTINCT Location
    FROM CourseDB.Location;

 * sqlite:///CourseDataWarehouse.db
0 rows affected.
207 rows affected.


[]

In [8]:
%%sql
DELETE FROM ProgramDimension;
INSERT INTO ProgramDimension(ProgramName, ProgramCode)
    SELECT DISTINCT ProgramName, ProgramCode
    FROM CourseDB.Program; 

 * sqlite:///CourseDataWarehouse.db
0 rows affected.
83 rows affected.


[]

In [9]:
%%sql
DELETE FROM TimecodeDimension;
INSERT INTO TimecodeDimension(Day,StartTime, EndTime)
    SELECT DISTINCT Day,substr(StartTime,12) AS StartTime, substr(EndTime,12) AS EndTime
    FROM CourseDB.Meeting; 

 * sqlite:///CourseDataWarehouse.db
0 rows affected.
1090 rows affected.


[]

In [10]:
%%sql
DROP TABLE IF EXISTS ClassTimecode;
CREATE TABLE ClassTimecode AS 
SELECT ClassID, TimecodeDimensionID
FROM CourseDB.Meeting JOIN TimecodeDimension ON (CourseDB.Meeting.Day = TimecodeDimension.Day AND substr(CourseDB.Meeting.StartTime,12) = TimecodeDimension.StartTime AND substr(CourseDB.Meeting.EndTime,12) = TimecodeDimension.EndTime);

 * sqlite:///CourseDataWarehouse.db
Done.
Done.


[]

In [11]:
%%sql
DELETE FROM CatalogCourseDimension;
INSERT INTO CatalogCourseDimension(CatalogCourseID, CatalogYear, CatalogID, ProgramID, Credits, Prereqs, Coreqs, Fees, Attributes, Description)
    SELECT DISTINCT CatalogCourseID, CatalogYear, CatalogID,ProgramID, Credits, Prereqs, Coreqs, Fees, Attributes, Description
    FROM CourseDB.CatalogCourse; 

 * sqlite:///CourseDataWarehouse.db
0 rows affected.
4440 rows affected.


[]

In [12]:
%%sql
DELETE FROM ClassFactTable;

INSERT INTO ClassFactTable(ClassID, ProfessorID,ProgramID,LocationID,TimecodeDimensionID,CatalogCourseID,Term,CRN,Section,Title,Credits,
                           Capacity,Actual,Remaining)
    SELECT DISTINCT Class.ClassID, Class.ProfessorID,ProgramDimension.ProgramID,LocationDimension.LocationID,
    ClassTimecode.TimecodeDimensionID,CatalogCourseDimension.CatalogCourseID,Class.Term,Class.CRN,Class.Section,Class.Title,Class.Credits,
    Class.Capacity,Class.Actual,Class.Remaining
    FROM CourseDB.Meeting AS Meeting
        JOIN CourseDB.Class AS Class ON (Meeting.ClassID = Class.ClassID)
        LEFT JOIN CatalogCourseDimension ON (Class.CatalogCourseID = CatalogCourseDimension.CatalogCourseID)
        LEFT JOIN ProgramDimension ON (ProgramDimension.ProgramCode = substr(CourseDB.Class.CatalogID,1,2))
        LEFT JOIN LocationDimension ON (LocationDimension.LocationID = Meeting.LocationID)
        LEFT JOIN ClassTimecode ON (Meeting.ClassID = ClassTimecode.ClassID)
        LEFT JOIN TimecodeDimension USING (TimecodeDimensionID);

 * sqlite:///CourseDataWarehouse.db
0 rows affected.
27286 rows affected.


[]

## __Integrity Check__

In [13]:
%%sql
SELECT
    (SELECT Count(*) FROM ProfessorDimension) as ProfessorDimension,
    (SELECT Count(*) FROM ProgramDimension) as ProgramDimension,
    (SELECT Count(*) FROM LocationDimension) as LocationDimension,
    (SELECT Count(*) FROM TimecodeDimension) as TimecodeDimension,
    (SELECT Count(*) FROM CatalogCourseDimension) as CatalogCourseDimension;

 * sqlite:///CourseDataWarehouse.db
Done.


ProfessorDimension,ProgramDimension,LocationDimension,TimecodeDimension,CatalogCourseDimension
1095,83,207,1090,4440


__Relational Integrity__

In [14]:
%%sql
SELECT CatalogCourseID, CatalogYear, Term, CRN, Section,CatalogCourseDimension.CatalogID as CatalogID, Title, Capacity, Actual, Remaining, substr(Term,-4) as Year
FROM ClassFactTable 
    LEFT JOIN ProfessorDimension USING (ProfessorID)
    LEFT JOIN CatalogCourseDimension USING (CatalogCourseID)
    LEFT JOIN ProgramDimension USING (ProgramID)
WHERE ProfessorDimension.Name like '%Huntley'
ORDER BY Year,Term DESC,CatalogID,Section;

 * sqlite:///CourseDataWarehouse.db
Done.


CatalogCourseID,CatalogYear,Term,CRN,Section,CatalogID,Title,Capacity,Actual,Remaining,Year
None,None,Fall2014,73060,A,None,Fundamentals of Web Design,25,26,-1,2014
None,None,Fall2014,73061,A,None,Systems Design and Implementation,25,16,9,2014
None,None,Fall2014,73061,A,None,Systems Design and Implementation,25,16,9,2014
None,None,Fall2014,70369,E,None,Intro to Information Systems,25,26,-1,2014
None,None,Fall2014,70369,E,None,Intro to Information Systems,25,26,-1,2014
None,None,Fall2015,76388,A,None,Fundamentals of Web Design,25,21,4,2015
None,None,Fall2015,76389,A,None,Systems Design and Implementation,25,13,12,2015
None,None,Fall2015,75231,E,None,Intro to Information Systems,29,28,1,2015
None,None,Fall2015,75231,E,None,Intro to Information Systems,29,28,1,2015
None,None,Fall2015,75246,F,None,Intro to Information Systems,29,28,1,2015


In [15]:
%%sql
SELECT Term, ClassID, Count(TimecodeDimensionID)
FROM ClassFactTable 
JOIN TimecodeDimension USING (TimecodeDimensionID)
WHERE CRN=39006 and Term = 'Spring2019'
GROUP BY ClassID;

 * sqlite:///CourseDataWarehouse.db
Done.


Term,ClassID,Count(TimecodeDimensionID)
Spring2019,13345,4


In [16]:
%%sql
SELECT ClassID, TimecodeDimensionID, StartTime
FROM ClassFactTable 
JOIN TimecodeDimension USING (TimecodeDimensionID)
WHERE CRN=39006 AND Term="Spring2019";

 * sqlite:///CourseDataWarehouse.db
Done.


ClassID,TimecodeDimensionID,StartTime
13345,31,18:30:00
13345,89,09:00:00
13345,31,18:30:00
13345,89,09:00:00


In [17]:
%%sql
SELECT *
FROM ClassFactTable
LIMIT 40;

 * sqlite:///CourseDataWarehouse.db
Done.


ClassID,ProfessorID,ProgramID,LocationID,TimecodeDimensionID,CatalogCourseID,Term,CRN,Section,Title,Credits,Capacity,Actual,Remaining
1,1,12,1,1,None,Fall2014,70384,C01,Introduction to Financial Accounting,3,0,31,-31
1,1,12,1,2,None,Fall2014,70384,C01,Introduction to Financial Accounting,3,0,31,-31
2,1,12,1,3,None,Fall2014,70385,C02,Introduction to Financial Accounting,3,0,31,-31
2,1,12,1,4,None,Fall2014,70385,C02,Introduction to Financial Accounting,3,0,31,-31
3,1,12,1,5,None,Fall2014,70382,C03,Introduction to Financial Accounting,3,0,31,-31
3,1,12,1,6,None,Fall2014,70382,C03,Introduction to Financial Accounting,3,0,31,-31
4,2,12,2,7,None,Fall2014,70291,C04,Introduction to Financial Accounting,3,0,29,-29
4,2,12,2,8,None,Fall2014,70291,C04,Introduction to Financial Accounting,3,0,29,-29
5,2,12,2,9,None,Fall2014,70350,C05,Introduction to Financial Accounting,3,0,30,-30
5,2,12,2,10,None,Fall2014,70350,C05,Introduction to Financial Accounting,3,0,30,-30


In [18]:
%%sql
DETACH DATABASE 'CourseDB';

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [19]:
%%sql
vacuum;

 * sqlite:///CourseDataWarehouse.db
Done.


[]